# Create layout based on cluster information
* Load network in GraphML
* Convert string group information to integers
* Apply

In [1]:
from graph_tool.all import *

#ORIGINAL = "nest2.graphml"

ORIGINAL = "nest2_large.graphml"
FILTERED = "nest-filtered.graphml" # Score > 0.6

In [2]:
nest = load_graph(ORIGINAL)

In [3]:
members = nest.vertex_properties["layoutMember"]

value_set = set()
for member in members:
    value_set.add(member)

v2idx = {}
idx = 1
for entry in value_set:
    v2idx[entry] = idx
    idx += 1

# Group names to integers
print(v2idx)

{'Group:NEST:19': 1, 'Group:NEST:6': 2, 'Group:NEST:35': 3, 'Group:NEST:24': 4, 'Group:NEST:145': 5, 'Group:NEST:15': 6, 'Group:NEST:116': 7, 'Group:NEST:7': 8, 'Group:NEST:12': 9, 'Group:NEST:18': 10, 'Group:NEST:13': 11, 'Group:NEST:14': 12, 'Group:NEST:1': 13, 'Group:NEST:25': 14, 'Group:NEST:5': 15, 'Group:NEST:26': 16, 'Group:NEST:97': 17, 'Group:NEST:10': 18, 'Group:NEST:9': 19, 'Group:NEST:11': 20, 'Group:NEST:16': 21, 'Group:NEST:33': 22, 'Group:NEST:2': 23, 'Group:NEST:17': 24, 'Group:NEST:23': 25, 'Group:NEST:4': 26, 'Group:NEST:108': 27, 'Group:NEST:217': 28, 'Group:NEST:3': 29, 'Group:NEST:8': 30}


In [4]:
# New attribute to show membership as an integer
groups = nest.new_vertex_property("int32_t")
nodes = nest.vertices()
for node in nodes:
    groups[node] = v2idx[members[node]]
    
# Name it "layoutIndex"
nest.vp.layoutIndex = groups

# Edge scores to be used for layout
score = nest.edge_properties["Score"]

In [5]:
# pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=0.01, mu=1000, eweight=score, C=0.5, multilevel=True)

# Dense cluster
#pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=0.01, mu=10000, eweight=score, C=0.5, multilevel=True)

# Clusters are closer to each other
#pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=10000, mu=10000, eweight=score, C=0.5, multilevel=True)

pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=10000, mu=10000, eweight=score, C=0.5)


# graph_draw(nest, pos=pos1, vertex_fill_color=groups, edge_color=[0.7, 0.7,0.7, 0.3], output="graph-draw-sfdp-all.pdf")

In [6]:
# Adjust position (For Cytoscape Desktop)
x = nest.new_vertex_property("double")
y = nest.new_vertex_property("double")

# Original positions are too dense for CYtoscape.
SCALING = 100

vs = nest.vertices()
for v in vs:
    p = pos1[v]
    x[v] = p[0] * SCALING
    y[v] = p[1] * SCALING

nest.vp.xpos = x
nest.vp.ypos = y

In [7]:
# list(nest.vertex_properties.keys())
nest.save("export-scaled-full-large.graphml", fmt="graphml")